In [1]:
pip install pandas numpy scikit-learn flask fastapi uvicorn

In [2]:
import pandas as pd
# Load the dataset
data = pd.read_csv('train.csv')
# Show the first few rows of the dataset
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
# Check for missing values
missing_data = data.isnull().sum()

# Fill missing numerical values with the median
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')
data_imputed = pd.DataFrame(imputer.fit_transform(data.select_dtypes(include=['float64', 'int64'])))

# Update the original dataset with imputed values
data.update(data_imputed)

# Convert categorical columns to numerical using pd.get_dummies
data_encoded = pd.get_dummies(data, drop_first=True)


In [5]:
from sklearn.preprocessing import StandardScaler

# Select features (drop 'SalePrice' as it's the target)
X = data_encoded.drop('SalePrice', axis=1)
y = data_encoded['SalePrice']

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [6]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
from sklearn.ensemble import RandomForestRegressor

# Initialize the model
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Evaluate the model
from sklearn.metrics import mean_squared_error

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mse


806323135.5086114

In [7]:
import pickle

# Save the trained model to a pickle file
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)


In [8]:
%%writefile app.py
from flask import Flask, request, jsonify
import pickle

app = Flask(__name__)

# Load the trained model
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json(force=True)
    features = data['features']
    prediction = model.predict([features])
    return jsonify({'prediction': prediction[0]})

if __name__ == '__main__':
    app.run(debug=True)


Writing app.py


In [12]:
import requests

# Define the URL of your running Flask app
url = 'http://127.0.0.1:5000/predict'

# Example feature input for the Boston Housing dataset (13 features)
sample_data = {
    "features": [0.02731, 0.0, 7.07, 0.0, 0.469, 6.421, 78.9, 4.9671, 2.0, 242.0, 17.8, 396.90, 9.14]
}

# Send POST request
response = requests.post(url, json=sample_data)

# Show the response
print("Status Code:", response.status_code)
print("Prediction:", response.json())


Status Code: 500


JSONDecodeError: Expecting value: line 1 column 1 (char 0)